# LangGraph Agent with Human-in-the-Loop

## Setup

In [ ]:

!pip install -U langgraph langchain langchain-core



In [ ]:
!pip install --force-reinstall langchain-core


In [56]:
from langchain_core.tools import BaseTool, tool as create_tool
from langchain_core.runnables import RunnableConfig


In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.types import interrupt, Command
print("hi")

In [ ]:
pip install -U langchain-anthropic

In [61]:
import os
os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGCHAIN_API_KEY_HERE"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"


In [ ]:
!pip uninstall regex -y
!pip install regex


In [ ]:
!pip uninstall tiktoken -y
!pip install tiktoken


## Define Tool with Interrupt

In [59]:
def schedule_meeting(person: str, date: str, time: str):   
    proposal = f"""
    Proposed Meeting:
    - With: {person}
    - On: {date}
    - At: {time}
    """
    # Pause here for human approval
    response = interrupt(
        f"Here is the proposed meeting schedule:\n{proposal}\n\nPlease approve or edit."
    )

    # Process human response
    if response["type"] == "accept":
        pass
    elif response["type"] == "edit":
        person = response["args"]["person"]
        date = response["args"]["date"]
        time = response["args"]["time"]
    else:
        raise ValueError(f"Unsupported response type: {response['type']}")

    return f" Meeting confirmed with {person} on {date} at {time}."


In [60]:
from langgraph.prebuilt import create_react_agent


In [66]:
from langgraph.types import interrupt

def schedule_meeting_with_approval(person: str, date: str, time: str):
   """Schedule a meeting with a person at a specific date and time."""

   response = interrupt(
       f"Schedule meeting with person='{person}', date='{date}', time='{time}'. Approve or edit?"
   )
   
   if response["type"] == "accept":      
       return f" Meeting confirmed with {person} on {date} at {time}."
   elif response["type"] == "edit":
       person = response["args"].get("person", person)
       date = response["args"].get("date", date)
       time = response["args"].get("time", time)
       return f"Meeting confirmed with {person} on {date} at {time}."
   else:
       return "Meeting scheduling cancelled or unknown response."


In [80]:
checkpointer = InMemorySaver()

agent = create_react_agent(
    model="openai:gpt-4",
    tools=[schedule_meeting_with_approval],

    checkpointer=checkpointer,
)




## Run Agent

In [81]:
config = {"configurable": {"thread_id": "meeting-123"}}

for chunk in agent.stream(
    {
        "messages": [{
            "role": "user",
            "content": "schedule_meeting with person='Max', date='Tuesday', time='3pm'"
        }]
    },
    config
):
    print(chunk)


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_B7cMYiT5WLsD2LJa80emH6gb', 'function': {'arguments': '{\n"person": "Max",\n"date": "Tuesday",\n"time": "3pm"\n}', 'name': 'schedule_meeting_with_approval'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 80, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BbSj3bonaA99RyXBtF4o28CWwC3IJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--41bfbaf0-17cb-4f5c-8d10-4c6aa4a1d1a7-0', tool_calls=[{'name': 'schedule_meeting_with_approval', 'args': {'person': 'Max', 'date': 'Tuesday', 'time': '3pm'}, 'id': 'call_B7cMYiT5WLsD2LJa80emH6gb', 'type': 'tool_ca

In [86]:
import textwrap

# Getting user input 
person = input("Enter person to meet: ").strip()
date = input("Enter meeting date: ").strip()
time = input("Enter meeting time: ").strip()

# Prepare user message for agent
user_message = f"schedule_meeting with person='{person}', date='{date}', time='{time}'"

interrupt_message = f"""
Here is the proposed meeting schedule:

    Proposed Meeting:
    - With: {person}
    - On: {date}
    - At: {time}

Please approve, edit, reject, or respond with feedback.
"""

print(textwrap.dedent(interrupt_message))

# Human-in-the-loop input
decision = input("Type your decision (accept / edit / reject / respond): ").strip().lower()

if decision == "accept":
    resume_payload = {"type": "accept"}

elif decision == "reject":
    resume_payload = {"type": "reject"}

elif decision == "edit":
    new_person = input(f"New person (leave blank to keep '{person}'): ").strip()
    new_date = input(f"New date (leave blank to keep '{date}'): ").strip()
    new_time = input(f"New time (leave blank to keep '{time}'): ").strip()

    args = {
        "person": new_person if new_person else person,
        "date": new_date if new_date else date,
        "time": new_time if new_time else time,
    }
    resume_payload = {"type": "edit", "args": args}

elif decision == "response":
    feedback = input("Enter your feedback or comment: ").strip()
    resume_payload = {"type": "response", "args": feedback}

else:
    print("Invalid input, rejecting by default.")
    resume_payload = {"type": "reject"}

# Output the choice
print("\nYour choice:", resume_payload)

# Handle decision
if resume_payload["type"] == "accept":
    print(f"\n Meeting confirmed with {person} on {date} at {time}.")

elif resume_payload["type"] == "reject":
    print("\n Meeting scheduling cancelled.")

elif resume_payload["type"] == "edit":
    updated = resume_payload["args"]
    print(f"\n Meeting updated to {updated['person']} on {updated['date']} at {updated['time']}.")

elif resume_payload["type"] == "response":
    print("\n Feedback received:")
    print(f"“{resume_payload['args']}”")


Enter person to meet:  Anvita
Enter meeting date:  27th May
Enter meeting time:  3pm



Here is the proposed meeting schedule:

    Proposed Meeting:
    - With: Anvita
    - On: 27th May
    - At: 3pm

Please approve, edit, reject, or respond with feedback.



Type your decision (accept / edit / reject / respond):  edit
New person (leave blank to keep 'Anvita'):  
New date (leave blank to keep '27th May'):  
New time (leave blank to keep '3pm'):  5pm



Your choice: {'type': 'edit', 'args': {'person': 'Anvita', 'date': '27th May', 'time': '5pm'}}

 Meeting updated to Anvita on 27th May at 5pm.
